In [2]:
from imp import reload
import LinearImageClassifier.FeatureMap as FeatureMap
reload(FeatureMap)
FeatureMap = FeatureMap.FeatureMap

import tensorflow as tf
import rasterio as rio
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as feature
import numpy as np
from skimage import img_as_ubyte
import matplotlib
import os
from rasterio.plot import show, show_hist

In [46]:
DATA_ROOT = "../data/"
imageindex_file = "train_v2.csv"
imagedir = 'train-tif-v2'
jpgdir = 'train-jpg'
img_template = "train_%d.%s"
imageindex = pd.read_csv(os.path.join(DATA_ROOT, imageindex_file))
labelTypes = set(" ".join(imageindex.tags.values).split(" "))
labelIDs = {l:i for l, i in zip(labelTypes, range(len(labelTypes)))}
print(labelIDs)

def _readimg(filename):
    return rio.open(filename).read()

def _onehot(labels, labelIDs):
    encoding = np.zeros(len(labelIDs))
    for label in labels.split(" "):
        encoding[labelIDs[label]] = 1
    return(np.array(encoding))
        
def imageparser(imgID, _):
    # input from tf dataset with (trainids, onehotlabels).map
    file = os.path.join(DATA_ROOT, imagedir, img_template %(imgID, 'tif'))
    data = rio.open(file).read()
    return(img_as_ubyte(data), _)
imageindex['onehot_tags'] = list(map(lambda row: _onehot(row[1].tags, labelIDs), imageindex.iterrows()))

{'slash_burn': 0, 'artisinal_mine': 1, 'primary': 2, 'road': 3, 'conventional_mine': 4, 'clear': 5, 'cloudy': 6, 'selective_logging': 7, 'blow_down': 8, 'habitation': 9, 'bare_ground': 10, 'agriculture': 11, 'blooming': 12, 'haze': 13, 'partly_cloudy': 14, 'water': 15, 'cultivation': 16}


In [68]:
trainids = imageindex.index.values
trainlabels = np.vstack(imageindex.onehot_tags.values)
dataset = tf.data.Dataset.from_tensor_slices((trainids[:-1], trainlabels[:-1]))

print(dataset)
dataset = dataset.map(
    lambda imgid, label: tuple(tf.py_func(
        imageparser, [imgid, label], [tf.uint8, tf.float64])))

iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)


<TensorSliceDataset shapes: ((), (17,)), types: (tf.int64, tf.float64)>


In [69]:
dataset

<MapDataset shapes: (<unknown>, <unknown>), types: (tf.uint8, tf.float64)>

In [77]:
import LinearImageClassifier.FeatureMap as FeatureMap
reload(FeatureMap)
FeatureMap = FeatureMap.FeatureMap
fm = FeatureMap(dataset, K=350, num_patches=20, patch_size=5, whiten=False)

In [78]:
fm

In [ ]:
fm.fit()




0it [00:00, ?it/s]/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()
/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from uint16 to uint8
  .format(dtypeobj_in, dtypeobj_out))



100it [00:00, 141.51it/s]


200it [00:01, 159.78it/s]


300it [00:01, 174.65it/s]


400it [00:02, 186.65it/s]


500it [00:02, 196.60it/s]


600it [00:02, 204.62it/s]


700it [00:03, 211.24it/s]


800it [00:03, 214.32it/s]


900it [00:04, 217.03it/s]


1000it [00:04, 220.22it/s]


1100it [00:05, 222.51it/s]


1200it [00:05, 222.82it/s]


1300it [00:06, 224.02it/s]


1400it [00:06, 224.68it/s]


1500it [00:06, 225.17it/s]


1600it [00:07, 225.13it/s]


1700it [00:08, 169.78it/s]


1800it [00:09, 142.00it/s]


1900it [00:10, 127.85it/s]


2000it [00:1

20200it [03:09, 100.91it/s]


20300it [03:10, 100.80it/s]


20400it [03:11, 100.21it/s]


20500it [03:12, 98.22it/s] 


20600it [03:13, 99.25it/s]


20700it [03:14, 99.43it/s]


20800it [03:15, 99.95it/s]


20900it [03:16, 101.67it/s]


21000it [03:17, 102.15it/s]


21100it [03:18, 97.79it/s] 


21200it [03:19, 97.75it/s]


21300it [03:20, 100.01it/s]


21400it [03:21, 99.57it/s] 


21500it [03:22, 98.54it/s]


21600it [03:23, 98.24it/s]


21700it [03:24, 100.05it/s]


21800it [03:25, 101.28it/s]


21900it [03:26, 100.05it/s]


22000it [03:27, 101.67it/s]


22100it [03:28, 102.54it/s]


22200it [03:29, 100.21it/s]


22300it [03:30, 100.53it/s]


22400it [03:31, 100.00it/s]


22500it [03:32, 100.52it/s]


22600it [03:33, 100.02it/s]


22700it [03:34, 99.81it/s] 


22800it [03:35, 98.89it/s]


22900it [03:36, 100.13it/s]


23000it [03:37, 98.61it/s] 


23100it [03:38, 97.23it/s]


23200it [03:40, 96.51it/s]


23300it [03:40, 98.94it/s]


23400it [03:41, 99.78it/s]


23500it [03:42, 100.9

In [76]:
len(fm.estimator.cluster_centers_)

350

In [39]:
len(fm.estimator.cluster_centers_[0])

75